# Проект: pretraining LLM и posttraining LLM

## Постановка задачи

- **Pretraining**
    - Претрейн — самый ресурсоёмкий этап обучения LLM. Чтобы полноценно обучить даже небольшую модель (менее 1B), понадобится более 10к GPU-часов на A100. Чтобы не тратить недели на обучение, но отработать ключевые приёмы, в проекте вы выполните упрощённую задачу. 
    - При полноценном претрейне модель учится обобщать знания из данных, на которых происходило обучение, чтобы потом извлекать эти знания по текстовым запросам уже после обучения. Упростим задачу — научим модель только структуре языка. 
    - Сосредоточимся на одном узком домене — **текстах произведений русской литературы** — и **обучим модель продолжать фразы из этого домена разумным текстом**. 
- **Posttraining**
    - Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель **Qwen2.5-0.5B**, с которой вы встречались в уроках. **Обучите её генерировать ответы на инструктивные русскоязычные вопросы**.

## Критерии успеха
- **Pretraining** - **чтобы оценить качество, используйте промпты**:
    - "Все мысли, которые имеют огромные последствия",
    - "Сила войска зависит от его духа",
    - "Мысль о том, что он принес страдания",
    - "Человек сознает себя свободным",
    - "Что бы ни случилось, я всегда буду",
    - "Любовь мешает смерти",
    - "Нет, жизнь не кончена",
    - "Всякая мысль, даже самая простая",
    - "Война не любезность, а самое гадкое дело",
    - "Чтобы жить честно"
- **Posttraining** - Факты могут быть ошибочными, но **язык ответа должен быть русским и должна сохраняться структура ответа**. Качество данных **должно быть сопоставимо с таким вариантом**:
    - Model Input 1:
    сколько планет в нашей солнечной системе?
        - Model Output 1:
    Согласно последним исследованиям, Солнце имеет 8 планеты. Это дает планетам, которые соответствуют разным критериям, таким как диета, местоположение, плотность, климат и другие.
    assistant
    Общее размер нашей Солнечной системы составляет примерно 9,9 миллиарда километров, а в результате на каждом из них есть планета,
    - Model Input 2:
    расскажи стих
        - Model Output 2:
    Вот сладкий сладкий вкус, который ты получаешь на себе
    Когда я говорю, что я люблю тебя
    Мое сердце, у меня есть все, что я хочу, чтобы сделать
    У меня есть все, что мне нужно, чтобы быть хорошим
    Так давай пойдем в ресторан
    assistant
    Пока мы уседим, что мы можем
    И в этот момент мы возвращаемся к при
    - Model Input 3:
    когда собирать крыжовник?
        - Model Output 3:
    Когда собираешь крыжовник, важно не беспокоиться о том, что другие могут его хвастаться. Вместо этого проверяйте свои способности и готовность. Вы можете изучить свой талант и стремления, прежде чем принимать решение, и признавать, что у вас есть свои сильные стороны, а также слабые места. Не снимайте крыжовник в обществе, и обязательно помните о
    - Model Input 4:
    Как быстро выучить новый язык?
        - Model Output 4:
    Скорость усвоения нового языка может сильно различаться в зависимости от нескольких факторов, таких как скорость, с которой вы пакетируете информацию через ее и ее способность учиться в различных условиях. Тем не менее, вот некоторые общие рекомендации: 1. Делайте сценарии. Обычно это может занять от 2 до 4 лет, чтобы понять основные концепции. Вы можете начать с

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных

- Pretrain
    - https://github.com/JoannaBy/RussianNovels/tree/master/corpus 
- Posttrain
    - русскоязычный инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) в диалоговом формате

## Инициализация

### Импорт библиотек

In [1]:
import os
import re
from git import Repo
from pathlib import Path
from datasets import Dataset, DatasetDict
import pandas as pd
from tokenizers import Tokenizer, pre_tokenizers, trainers
from transformers import (
    LlamaConfig,
    LlamaForCausalLM,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    DataCollatorForLanguageModeling
)
from tokenizers.models import BPE
from transformers import LlamaConfig, LlamaForCausalLM
from transformers import Trainer, TrainingArguments
import torch
import accelerate

C:\Users\Papa\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1029 22:57:21.545000 22360 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


### Установка главных параметров

In [2]:
PRETRAIN_DATASET =  "pretrain-dataset"
DICT_SIZE = 3000
SPECIAL_TOKEN = "<|endoftext|>"
CONTEXT_LENGTH = 512

# Pretraining

<div class="alert alert-secondary">


Скачайте данные из репозитория и упакуйте их в один датасет. Вам понадобятся все произведения из репозитория.

Проведите препроцессинг данных:
- Очистите их от дубликатов.
- Очистите от предложений с буквами не из кириллицы.
- Обработайте повторяющуюся пунктуацию и т. д.
- Разбейте на чанки поменьше, чтобы можно было добавить <bos> и <eos> токены в соответствии с обучаемой длиной контекста.

In [3]:
if not os.path.isdir(PRETRAIN_DATASET):
    Repo.clone_from("https://github.com/JoannaBy/RussianNovels.git", PRETRAIN_DATASET) 

In [4]:
texts = []
for file_path in Path(Path(PRETRAIN_DATASET, "corpus")).glob("*.txt"):
    with file_path.open("r", encoding="utf-8") as f:
        for line in f:
            texts.append(line.strip())

# удалим строки с недопустимыми символами
allowed_pattern = re.compile(r"^[А-Яа-яЁё0-9.,!?;:\-()\"'\s]+$")
texts = [t for t in texts if allowed_pattern.fullmatch(t)]

# Заменяем повторяющиеся знаки пунктуации на один
texts = [re.sub(r'([.,!?;:\-()])\1+', r'\1', t) for t in texts]

#удалим дубликаты
texts = pd.DataFrame({'t': texts}).drop_duplicates()['t'].tolist()

#перенесём строки, если они длиннее 300 слов
max_words = 300 # из рассчёта, что ~1.5 токена на слово, а токенов у нас в контексте 512
result = []
for text in texts:
    words = text.split()
    if len(words) > max_words:
        parts = [' '.join(words[i:i + max_words]) \
                    for i in range(0, len(words), max_words)]
        result.extend(parts)
    else:
        result.append(text)
texts = result

In [5]:
TEST_TEXT = texts[100]
TEST_TEXT

'- Тяжкое, тяжкое время, что говорить, - пробормотал он, - но унывать-то'

<div class="alert alert-secondary">

Создайте и обучите собственный токенизатор на полученных данных. Размер словаря выберите небольшим: при обучении только на рассмотренных текстах — около 3к токенов. В рассматриваемых данных язык намного менее разнообразен, чем в совокупных данных, поэтому крупные токенизаторы от реальных LLM могут не подойти. 
При создании токенизатора можете ориентироваться на [материал huggingface.co](https://huggingface.co/learn/llm-course/ru/chapter6/8). Рекомендуем использовать BPE.

In [6]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer.pre_tokenize_str(TEST_TEXT)

[('-', (0, 1)),
 ('ĠÐ¢ÑıÐ¶ÐºÐ¾Ðµ', (1, 8)),
 (',', (8, 9)),
 ('ĠÑĤÑıÐ¶ÐºÐ¾Ðµ', (9, 16)),
 ('ĠÐ²ÑĢÐµÐ¼Ñı', (16, 22)),
 (',', (22, 23)),
 ('ĠÑĩÑĤÐ¾', (23, 27)),
 ('ĠÐ³Ð¾Ð²Ð¾ÑĢÐ¸ÑĤÑĮ', (27, 36)),
 (',', (36, 37)),
 ('Ġ-', (37, 39)),
 ('ĠÐ¿ÑĢÐ¾Ð±Ð¾ÑĢÐ¼Ð¾ÑĤÐ°Ð»', (39, 51)),
 ('ĠÐ¾Ð½', (51, 54)),
 (',', (54, 55)),
 ('Ġ-', (55, 57)),
 ('ĠÐ½Ð¾', (57, 60)),
 ('ĠÑĥÐ½ÑĭÐ²Ð°ÑĤÑĮ', (60, 68)),
 ('-', (68, 69)),
 ('ÑĤÐ¾', (69, 71))]

In [7]:
def get_training_corpus(texts):
    for i in range(0, len(texts), 1000):
        batch = [texts[j] for j in range(i, min(i + 1000, len(texts)))]
        yield batch

In [8]:
%%time
trainer = trainers.BpeTrainer(vocab_size=DICT_SIZE, special_tokens=[SPECIAL_TOKEN])
tokenizer.train_from_iterator(get_training_corpus(texts), trainer=trainer)

CPU times: total: 2min 6s
Wall time: 25.2 s


Посмотрим на токенизатор:

In [9]:
print("Vocab size:", tokenizer.get_vocab_size())
print("Sample vocab:", list(tokenizer.get_vocab().keys())[:30])


Vocab size: 3000
Sample vocab: ['ĠÐŁÑĢ', 'ĠÐ¿Ð¾Ð¹', 'ĠÐ¾Ð±ÐµÑī', 'ĠÐ¼ÐµÑĤ', 'ĠÑĢÐµÐ²', 'ĠÑĤÐ°Ð¼', 'ĠÑĤÐ¾Ð»ÑĮÐºÐ¾', 'ĠÐ¸Ð½', '!"', 'ĠÐ¾Ð±ÑĢÐ°Ñī', 'Ð½Ð¸ÐµÐ¼', 'ÐºÐ°Ð¼', 'ĠÑĩÐ°ÑģÑĤÐ¾', 'ÐµÐ½Ð½ÑĥÑİ', 'Ð°Ð¿', 'ĠÐ¾Ð±Ðµ', 'ĠÑĥÐ²Ð¸Ð´ÐµÐ»', 'Ð°Ð²ÑĪÐ¸Ð¹', 'ĠÐ³Ð¾Ð²Ð¾ÑĢÐ¸ÑĤÑĮ', 'ĠÑĥÑģÐ¿Ð¾ÐºÐ¾', 'ĠÐ³Ð¾Ð²Ð¾ÑĢÑİ', 'ĠÐ¾ÑĤÐºÑĢÑĭ', 'Ð½ÐµÑģ', 'ĠÑĢÑĥÐºÐ°Ñħ', '0', 'ĠÐ¿Ð¾Ð±', 'ÑĨÐ¸Ð¾Ð½', 'ÑĥÐ¹ÑĤÐµ', 'Ð¸Ñı', 'Ñĩ']


Посмотрим на токенизированный текст:

In [10]:
print (TEST_TEXT)
encoding = tokenizer.encode(TEST_TEXT)
print (encoding.tokens)

- Тяжкое, тяжкое время, что говорить, - пробормотал он, - но унывать-то
['-', 'ĠÐ¢', 'ÑıÐ¶', 'ÐºÐ¾Ðµ', ',', 'ĠÑĤÑıÐ¶', 'ÐºÐ¾Ðµ', 'ĠÐ²ÑĢÐµÐ¼Ñı', ',', 'ĠÑĩÑĤÐ¾', 'ĠÐ³Ð¾Ð²Ð¾ÑĢÐ¸ÑĤÑĮ', ',', 'Ġ-', 'ĠÐ¿ÑĢÐ¾Ð±', 'Ð¾ÑĢ', 'Ð¼Ð¾ÑĤÐ°Ð»', 'ĠÐ¾Ð½', ',', 'Ġ-', 'ĠÐ½Ð¾', 'ĠÑĥ', 'Ð½Ñĭ', 'Ð²', 'Ð°ÑĤÑĮ', '-', 'ÑĤÐ¾']


Посмотрим на 3й токен:

In [11]:
start, end = encoding.offsets[5]
TEST_TEXT[start:end]

' тяж'

In [12]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token=SPECIAL_TOKEN,
    eos_token=SPECIAL_TOKEN,
    unk_token="<unk>",
    pad_token=SPECIAL_TOKEN # Зададим пэддинг-токен как токен конца
    ) 

<div class="alert alert-secondary">

Токенизируйте данные и подготовьте их к претрейну с длиной контекста 512 токенов в виде экземпляра класса transformers.Dataset.

In [13]:
texts = [text + SPECIAL_TOKEN for text in texts]

In [14]:
texts[:5]

['Посвящается Любови Евгеньевне Белозерской<|endoftext|>',
 'Пошел мелкий снег и вдруг  повалил  хлопьями.<|endoftext|>',
 'Ветер завыл; сделалась метель. В одно  мгновение<|endoftext|>',
 'темное  небо  смешалось  с  снежным  морем.  Все<|endoftext|>',
 'исчезло.<|endoftext|>']

In [ ]:
%%time
encodings = wrapped_tokenizer(
    texts,
    add_special_tokens=False,
    truncation=False,
)
all_ids = []
for ids in encodings["input_ids"]:
    all_ids.extend(ids)

chunks = [
    all_ids[i:i + CONTEXT_LENGTH]
    for i in range(0, len(all_ids), CONTEXT_LENGTH)
    if len(all_ids[i:i + CONTEXT_LENGTH]) == CONTEXT_LENGTH
]

attention_masks = [[1] * CONTEXT_LENGTH for _ in chunks]

dataset = Dataset.from_dict({
    "input_ids": chunks,
    "attention_mask": attention_masks,
})

dataset

CPU times: total: 15.6 ms
Wall time: 52 ms


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4
})

<div class="alert alert-secondary">

Инициализируйте модель ~150M параметров c произвольной decoder-only архитектурой трансформера. Например, можно рассмотреть LlamaConfig с параметрами:
- hidden_size=1024, intermediate_size=1536, num_hidden_layers=16, num_attention_heads=16, num_key_value_heads=8.

Подготовьте коллбэки для валидации качества на промптах. Реализуйте обучение с помощью Trainer. Обратите внимание на параметры регуляризации weight_decay. Используйте подходящий batch_size — в диапазоне 64—128.

In [19]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

In [20]:
config = LlamaConfig(
    vocab_size=wrapped_tokenizer.vocab_size,
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=512,
    bos_token_id=wrapped_tokenizer.bos_token_id,
    eos_token_id=wrapped_tokenizer.eos_token_id,
)

model = LlamaForCausalLM(config)
print(f"Модель инициализирована: {sum(p.numel() for p in model.parameters())/1e6:.1f}M параметров")

dataset = dataset.map(lambda x: {"labels": x["input_ids"]})

dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

collator = DataCollatorForLanguageModeling(
    tokenizer=wrapped_tokenizer,
    mlm=False
)

class PromptEvalCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts):
        self.tokenizer = tokenizer
        self.prompts = prompts

    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs["model"]
        model.eval()
        print("\n=== Валидация на промптах ===")
        for prompt in self.prompts:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=30)
            text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\n[Prompt] {prompt}\n[Output] {text}")

callback = PromptEvalCallback(wrapped_tokenizer, test_prompts)

training_args = TrainingArguments(
    output_dir="./llama-small-pretrain",
    per_device_train_batch_size=64,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=50,
    #evaluation_strategy="steps",
    do_eval=True,
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
    bf16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=wrapped_tokenizer,
    callbacks=[callback],
)

trainer.train()

Модель инициализирована: 132.0M параметров


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Papa\AppData\Roaming\Python\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 1/1 [00:10<00:00, 10.76s/it]

{'train_runtime': 10.769, 'train_samples_per_second': 0.279, 'train_steps_per_second': 0.093, 'train_loss': 8.186291694641113, 'epoch': 1.0}


TrainOutput(global_step=1, training_loss=8.186291694641113, metrics={'train_runtime': 10.769, 'train_samples_per_second': 0.279, 'train_steps_per_second': 0.093, 'train_loss': 8.186291694641113, 'epoch': 1.0})

Сгенерируйте ответы на запросы test_prompts. Чтобы ревьюер мог оценить результаты, оставьте генерацию в ноутбуке.

In [23]:
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

for prompt in test_prompts:
    inputs = wrapped_tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Удаляем token_type_ids если они есть
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=wrapped_tokenizer.pad_token_id,
            eos_token_id=wrapped_tokenizer.eos_token_id,
        )

    text = wrapped_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Prompt: {prompt}\nОтвет модели: {text}")

Prompt: Все мысли, которые имеют огромные последствия
Ответ модели: ÐĴ ÑģÐµ ĠÐ¼ÑĭÑģÐ»Ð¸, ĠÐºÐ¾ÑĤÐ¾ÑĢÑĭÐµ ĠÐ¸Ð¼ Ðµ ÑİÑĤ ĠÐ¾Ð³ÑĢÐ¾Ð¼ Ð½ÑĭÐµ ĠÐ¿Ð¾ÑģÐ»ÐµÐ´ ÑģÑĤÐ²Ð¸Ñı ÐµÑĢÐ½Ð° ÐµÑĢÐ¿ ÐµÐ¿ÐµÑĢÑĮ ĠÑģÐ²Ð¾ÐµÐ¼ ÑģÑĤÐ²Ð¾ ĠÑģÐ¾ Ð¼Ð¾Ð¶ ĠÑĢÐ°ÑģÐ¿ ĠÐ¾Ð±ÑĢÐ°Ð· ĠÐ´ÐµÐ»Ð¾ ÐµÐ½Ð½ÑĭÑħ Ð°Ñı Ð¼Ð¾Ð¶ ĠÐ±ÑĭÑģÑĤÑĢÐ¾ Ð¾Ð½ÑĮ ĠÑģÐ¾ ĠÑģÐºÐ¾Ð»ÑĮÐºÐ¾ Ġ Ġ ĠÐ¾Ð³Ð»Ñı ÐµÐ´ Ð»ÐµÐ½Ð°?" ĠÑģÐµÐ³Ð¾Ð´Ð½Ñı ÑģÑĤÐ¸ ÐµÐ¿ÐµÑĢÑĮ?" ĠÐ½Ð°ÐºÐ»Ð¾Ð½ ÑĢÐ¾Ð½ ÐºÐ°Ñı ĠÑģÐºÐ¾Ð»ÑĮÐºÐ¾ Ð¾Ð»Ðº ÐµÐ¿ÐµÑĢÑĮ ÑĥÑİÑī Ð¾Ð»Ðº ĠÑģÐºÐ¾Ð»ÑĮÐºÐ¾ ÑĢÑĭÑĤ ÑıÐ»Ð°ÑģÑĮ ĠÑģÐ¾ ÐµÐ´ Ð¾Ð»Ðº ĠÑĩÑĢÐµÐ· Ð½Ð¸ Ð¾Ð´Ð° ÐµÑģÑĮ ÑıÐ»Ð°ÑģÑĮ Ġ ĠÐ¾Ð±ÑĢÐ°Ð· ĠÑĪÑĥÑĤ ÐµÐ¿ÐµÑĢÑĮ
Prompt: Сила войска зависит от его духа
Ответ модели: Ð¡ Ð¸Ð»Ð° ĠÐ²Ð¾Ð¹ ÑģÐºÐ° ĠÐ·Ð°Ð² Ð¸Ñģ Ð¸ÑĤ ĠÐ¾ÑĤ ĠÐµÐ³Ð¾ ĠÐ´ÑĥÑħ Ð° ĠÐ³ÑĢÐ¾Ð¼ÐºÐ¾ ÑĤÑĭ Ð»Ð¾Ð² ÐµÐµ ÑİÑĤ ĠÐ¼ÐµÑĢÑĤÐ² ÐµÐµ ĠÐ±, Ð¾Ð¹ ÑĢÐ°Ð» ĠÑģÐ²Ð¾Ð¸Ñħ, Ġ ĠÐ²ÐµÐ»Ð¸ ĠÑģÑĤÑĭÐ´ Ð¾Ð½Ðµ ĠÐļÑĤÐ¾ ĠÐ¾ÑĤÐº ĠÐ½Ð°Ð± ĠÑĤÐ¸ ĠÐ²ÑĤÐ¾ÑĢ Ð½Ðµ Ð¾Ð¹ Ð¾ÑģÐºÐ² ĠÑģÐ»Ð¾Ð²Ð° ĠÑĤÐ¸ ĠÐ²ÑģÐµÑħ Ð»Ñı ĠÐļÑĤÐ¾ ĠÑģÐ¸Ð´ ĠÐ¡ÑĤÐ°ÑĢ ĠÐ´Ð¾Ð¶Ð´ ĠÐºÐ¾Ð¼Ð°Ð½Ð´ ĠÐ¾ÑĤÐº ĠÑģÐ»Ð¸ÑĪÐºÐ¾Ð¼ ĠÐ¾ÑĤÐº Ġ Ġ

# Post-train SFT

Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель Qwen2.5-0.5B, с которой вы встречались в уроках. Обучите её генерировать ответы на инструктивные русскоязычные вопросы.

Для оценки качества используйте такой набор вопросов:


In [21]:
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
  ]

1. Подготовьте инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) для обучения базовой модели. Представьте его в виде transformers.Dataset для диалоговых данных input=system, instruction=user, output=assistant.
1. С помощью trl SFTTrainer дообучите модель.
1. Посмотрите на адекватность генерации на финальных данных. 

Факты могут быть ошибочными, но язык ответа должен быть русским и должна сохраняться структура ответа.